# Creating an Einstein Toolkit thorn:
## The scalar wave equation

The goal of this tutorial is to construct an Einstein Toolkit (ETK) thorn that will evolve some initial data according to the scalar wave equation, subject to appropriate boundary conditions. This thorn should function identically to the $\text{WaveToy}$ and $\text{IDScalarWaveC}$ thorns included in the ETK.

### Evolving the Scalar Wave equation

After importing the core modules, we will set $\text{GridFuncMemAccess}$ to $\text{ETK}$. The scalar wave right-hand sides and plane wave initial data are already built by [$\text{Tutorial-ScalarWave.ipynb}$](Tutorial-ScalarWave.ipynb), so we can simply import them to use here. We will also need to instruct the system to create the directories for our thorn.

We will begin the the evolution equations.

In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

par.set_parval_from_str("grid::GridFuncMemAccess","ETK")

# Step 1b: Call the ScalarWave_RHSs() function from within the
#         ScalarWave/ScalarWave_RHSs.py module,
#         which should do exactly the same as in Steps 1-10 above.
import ScalarWave.ScalarWave_RHSs as swrhs
swrhs.ScalarWave_RHSs()

!mkdir WaveToyNRPy     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir WaveToyNRPy/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.


Now, we need to output these expressions to C code. We will do this the same way as in the Weyl scalars ETK tutorial: export these to a .h file, which can then be included in a main .c file to do the calculations.

In [2]:
# First, we will write the right-hand sides.
uu_rhs,vv_rhs = gri.register_gridfunctions("EVOL",["uu_rhs","vv_rhs"])
scalar_RHSs_to_print = [\
                        lhrh(lhs=gri.gfaccess("out_gfs","uu_rhs"),rhs=swrhs.uu_rhs),\
                        lhrh(lhs=gri.gfaccess("out_gfs","vv_rhs"),rhs=swrhs.vv_rhs),]
scalar_RHSs_CcodeKernel = fin.FD_outputC("returnstring",scalar_RHSs_to_print)

scalar_RHSs_looped = loop.loop(["i2","i1","i0"],["1","1","1"],["cctk_lsh[2]-1","cctk_lsh[1]-1","cctk_lsh[0]-1"],\
                               ["1","1","1"],["#pragma omp parallel for","",""],"",scalar_RHSs_CcodeKernel)

with open("WaveToyNRPy/src/ScalarWave_RHSs.h", "w") as file:
    file.write(str(scalar_RHSs_looped))


We will need to write C code to call these header files we have output. To do this, we can follow the examples of WaveToyC and our Weyl Scalar ETK thorn.

In [3]:
%%writefile WaveToyNRPy/src/WaveToyNRPy.c
#include <stddef.h>
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

void WaveToyNRPy_set_rhs(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  
#include "ScalarWave_RHSs.h" 
}

/* Boundary Condition code adapted from WaveToyC thorn in ETK, implementing built-in
 * ETK BC functionality
 */
void WaveToyNRPy_Boundaries(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  const char *bctype;


  bctype = NULL;
  if (CCTK_EQUALS(bound,"flat") || CCTK_EQUALS(bound,"static") ||
      CCTK_EQUALS(bound,"radiation") || CCTK_EQUALS(bound,"robin") ||
      CCTK_EQUALS(bound,"none"))
  {
    bctype = bound;
  }
  else if (CCTK_EQUALS(bound,"zero"))
  {
    bctype = "scalar";
  }

  /* Uses all default arguments, so invalid table handle -1 can be passed */
  if (bctype && (Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "WaveToyNRPy::uuGF", bctype) < 0 || 
                 Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "WaveToyNRPy::vvGF", bctype) < 0))
  {
    CCTK_WARN (0, "ScalarWave_Boundaries: Error selecting boundary condition");
  }
}

void WaveToyNRPy_InitSymBound(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
      
  int sym[3];

  sym[0] = 1;
  sym[1] = 1;
  sym[2] = 1;

  SetCartSymVN(cctkGH, sym,"WaveToyNRPy::uuGF");
  SetCartSymVN(cctkGH, sym,"WaveToyNRPy::vvGF");

  return;
}

void WaveToyNRPy_RegisterVars(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  CCTK_INT ierr CCTK_ATTRIBUTE_UNUSED = 0;
  /* Register all the evolved grid functions with MoL */
  ierr += MoLRegisterEvolved(CCTK_VarIndex("WaveToyNRPy::uuGF"),  CCTK_VarIndex("WaveToyNRPy::uu_rhsGF"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("WaveToyNRPy::vvGF"),  CCTK_VarIndex("WaveToyNRPy::vv_rhsGF"));
  /* Register all the evolved Array functions with MoL */
  return;
}

Overwriting WaveToyNRPy/src/WaveToyNRPy.c


We will start with the file $\text{interface.ccl}$. This file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist within those functions. Then, we tell the toolkit that we want the scalars $\text{uuGF}$ and $\text{vvGF}$ to be visible to other thorns by using the keyword "public". 

In [4]:
%%writefile WaveToyNRPy/interface.ccl
implements: WaveToyNRPy

inherits: Boundary grid

USES INCLUDE: loopcontrol.h
USES INCLUDE: Symmetry.h
USES INCLUDE: Boundary.h
    
CCTK_INT FUNCTION MoLRegisterEvolved(CCTK_INT IN EvolvedIndex, CCTK_INT IN RHSIndex)
USES FUNCTION MoLRegisterEvolved

SUBROUTINE Diff_coeff(CCTK_POINTER_TO_CONST IN cctkGH, CCTK_INT IN dir, CCTK_INT IN nsize, CCTK_INT OUT ARRAY imin, CCTK_INT OUT ARRAY imax, CCTK_REAL OUT ARRAY q, CCTK_INT IN table_handle)
USES FUNCTION Diff_coeff

CCTK_INT FUNCTION MultiPatch_GetMap(CCTK_POINTER_TO_CONST IN cctkGH)
USES FUNCTION MultiPatch_GetMap

CCTK_INT FUNCTION MultiPatch_GetBbox(CCTK_POINTER_TO_CONST IN cctkGH, CCTK_INT IN size, CCTK_INT OUT ARRAY bbox)
USES FUNCTION MultiPatch_GetBbox

CCTK_INT FUNCTION GetBoundarySpecification(CCTK_INT IN size, CCTK_INT OUT ARRAY nboundaryzones, CCTK_INT OUT ARRAY is_internal, CCTK_INT OUT ARRAY is_staggered, CCTK_INT OUT ARRAY shiftout)
USES FUNCTION GetBoundarySpecification

CCTK_INT FUNCTION MultiPatch_GetBoundarySpecification(CCTK_INT IN map, CCTK_INT IN size, CCTK_INT OUT ARRAY nboundaryzones, CCTK_INT OUT ARRAY is_internal, CCTK_INT OUT ARRAY is_staggered, CCTK_INT OUT ARRAY shiftout)
USES FUNCTION MultiPatch_GetBoundarySpecification

CCTK_INT FUNCTION SymmetryTableHandleForGrid(CCTK_POINTER_TO_CONST IN cctkGH)
USES FUNCTION SymmetryTableHandleForGrid

CCTK_INT FUNCTION Boundary_SelectGroupForBC(CCTK_POINTER_TO_CONST IN GH, CCTK_INT IN faces, CCTK_INT IN boundary_width, CCTK_INT IN table_handle, CCTK_STRING IN group_name, CCTK_STRING IN bc_name)
USES FUNCTION Boundary_SelectGroupForBC

CCTK_INT FUNCTION Boundary_SelectVarForBC(CCTK_POINTER_TO_CONST IN GH, CCTK_INT IN faces, CCTK_INT IN boundary_width, CCTK_INT IN table_handle, CCTK_STRING IN var_name, CCTK_STRING IN bc_name)
USES FUNCTION Boundary_SelectVarForBC

public:
cctk_real scalar_fields_rhs type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  uu_rhsGF,vv_rhsGF
} "The evolved scalar fields"

public:
cctk_real scalar_fields type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  uuGF,vvGF
} "The evolved scalar fields"


Overwriting WaveToyNRPy/interface.ccl


We will now write the file $\text{param.ccl}$. This file allows the listed parameters to be set at runtime. We also give allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3). We first set a Keyword to determine our boundary conditions; this is not implemented yet, but more options can be added later. We also set the number of timelevels we will store in memory.

In [5]:
%%writefile WaveToyNRPy/param.ccl
shares: MethodOfLines

USES CCTK_INT MoL_Num_Evolved_Vars
USES CCTK_INT MoL_Num_ArrayEvolved_Vars

restricted:
CCTK_INT SimpleWave_MaxNumEvolvedVars "Number of evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_Evolved_Vars STEERABLE=RECOVER
{
  2:2 :: "Number of evolved variables used by this thorn"
} 2

restricted:
CCTK_INT SimpleWave_MaxNumArrayEvolvedVars "Number of Array evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_ArrayEvolved_Vars STEERABLE=RECOVER
{
  0:0 :: "Number of Array evolved variables used by this thorn"
} 0

restricted:
KEYWORD bound "Type of boundary condition to use"
{
  "flat"      :: "Flat (von Neumann, n grad phi = 0) boundary condition"
  "static"    :: "Static (Dirichlet, dphi/dt=0) boundary condition"
  "radiation" :: "Radiation boundary condition"
  "robin"     :: "Robin (phi(r) = C/r) boundary condition"
  "zero"      :: "Zero (Dirichlet, phi=0) boundary condition"
  "none"      :: "Apply no boundary condition"
} "none"

restricted:
CCTK_INT timelevels "Number of active timelevels" STEERABLE=RECOVER
{
  0:3 :: ""
} 3

restricted:
CCTK_REAL wavespeed "The speed at which the wave propagates"
{
 *:* :: "Wavespeed as a multiple of c"
} 1.0

Overwriting WaveToyNRPy/param.ccl


Finally, we will write the file $\text{schedule.ccl}$; its official documentation is found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). This file dictates when the various parts of the thorn will be run. We first assign storage for both scalar gridfunctions, and then specify that we want our code run 

In [6]:
%%writefile WaveToyNRPy/schedule.ccl
STORAGE: scalar_fields_rhs[timelevels]
STORAGE: scalar_fields[timelevels]

schedule WaveToyNRPy_InitSymBound at BASEGRID
{
  LANG: C
  OPTIONS: global
} "Schedule symmetries"

schedule WaveToyNRPy_set_rhs as WaveToy_Evolution at EVOL
{
  LANG: C
  READS: uuGF(Everywhere)
  READS: vvGF(Everywhere)
  WRITES: uu_rhsGF(Interior)
  WRITES: VV_rhsGF(Interior)
} "Evolution of 3D wave equation"

schedule WaveToyNRPy_Boundaries as WaveToy_Boundaries at EVOL AFTER WaveToy_Evolution
{
  LANG: C
  OPTIONS: level
} "Boundaries of 3D wave equation"

schedule GROUP ApplyBCs as WaveToyNRPy_ApplyBCs at POSTRESTRICT after WaveToy_Boundaries
{
} "Apply boundary conditions"

schedule WaveToyNRPy_RegisterVars in MoL_Register
{
  LANG: C
  OPTIONS: meta
} "Register Variables for MoL"

schedule group ApplyBCs as WaveToyNRPy_ApplyBCs in MoL_PostStep after WaveToy_Boundaries
{
} "Apply boundary conditions controlled by thorn Boundary"


Overwriting WaveToyNRPy/schedule.ccl


We will also need $\text{make.code.defn}$, which states which files need to be compiled. This thorn only has the one C file to compile so far.

In [7]:
%%writefile WaveToyNRPy/src/make.code.defn
SRCS = WaveToyNRPy.c

Overwriting WaveToyNRPy/src/make.code.defn


### Scalar Wave Initial Data

Now, we will move on to a second thorn. This thorn will set the initial data.

We will now output the C code for the header file to set plane wave initial data that will be evolved by the first thorn we made in this tutorial.

In [8]:
# Step 1c: Call the InitialData_PlaneWave() function from within the
#         ScalarWave/InitialData_PlaneWave.py module,
#         which should do exactly the same as in Steps 1-5 above.
import ScalarWave.InitialData_PlaneWave as swid
x,y,z = gri.register_gridfunctions("AUX",["x","y","z"])
gri.xx[0] = x
gri.xx[1] = y
gri.xx[2] = z

swid.InitialData_PlaneWave()

!mkdir IDScalarWaveNRPy     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir IDScalarWaveNRPy/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Second, we will write the initial data.
scalar_PWID_to_print = [\
                        lhrh(lhs=gri.gfaccess("out_gfs","uu"),rhs=swid.uu_ID),\
                        lhrh(lhs=gri.gfaccess("out_gfs","vv"),rhs=swid.vv_ID),]
scalar_PWID_CcodeKernel = fin.FD_outputC("returnstring",scalar_PWID_to_print)

scalar_PWID_looped = loop.loop(["i2","i1","i0"],["1","1","1"],["cctk_lsh[2]-1","cctk_lsh[1]-1","cctk_lsh[0]-1"],\
                               ["1","1","1"],["#pragma omp parallel for","",""],"",\
                               scalar_PWID_CcodeKernel.replace("time","cctk_time"))

with open("IDScalarWaveNRPy/src/ScalarWave_PWID.h", "w") as file:
    file.write(str(scalar_PWID_looped))


We will write another C file with the functions we need here.

In [9]:
%%writefile IDScalarWaveNRPy/src/InitialData.c
#include <math.h>

#include "cctk.h"
#include "cctk_Parameters.h"
#include "cctk_Arguments.h"

void IDScalarWaveNRPy_InitialData(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS
  DECLARE_CCTK_PARAMETERS
  
  const CCTK_REAL *xGF = x;
  const CCTK_REAL *yGF = y;
  const CCTK_REAL *zGF = z;
#include "ScalarWave_PWID.h"
}

Overwriting IDScalarWaveNRPy/src/InitialData.c


We will also need to write an interface file for this thorn.

In [10]:
%%writefile IDScalarWaveNRPy/interface.ccl
implements: IDScalarWaveNRPy
inherits: WaveToyNRPy grid


Overwriting IDScalarWaveNRPy/interface.ccl


We will need to write a param file for this thorn.

In [11]:
%%writefile IDScalarWaveNRPy/param.ccl
shares: grid

USES KEYWORD type

shares: WaveToyNRPy
    
USES REAL wavespeed

restricted:
CCTK_KEYWORD initial_data "Type of initial data"
{
  "plane"      :: "Plane wave"
} "plane"

restricted:
CCTK_REAL kk0 "The wave number in the x-direction"
{
 *:* :: "No restriction"
} 4.0

restricted:
CCTK_REAL kk1 "The wave number in the y-direction"
{
 *:* :: "No restriction"
} 0.0

restricted:
CCTK_REAL kk2 "The wave number in the z-direction"
{
 *:* :: "No restriction"
} 0.0


Overwriting IDScalarWaveNRPy/param.ccl


We will need to write a schedule file for this thorn.

In [12]:
%%writefile IDScalarWaveNRPy/schedule.ccl
schedule IDScalarWaveNRPy_InitialData at CCTK_INITIAL as WaveToy_InitialData
{
  STORAGE:       WaveToyNRPy::scalar_fields[3]
  LANG:          C
  WRITES: uuGF(Everywhere)
  WRITES: vvGF(Everywhere)
} "Initial data for 3D wave equation"


Overwriting IDScalarWaveNRPy/schedule.ccl


We will also need $\text{make.code.defn}$, which states which files need to be compiled. This thorn only has the one C file to compile so far.

In [13]:
%%writefile IDScalarWaveNRPy/src/make.code.defn
SRCS = InitialData.c

Overwriting IDScalarWaveNRPy/src/make.code.defn
